In [1]:
#import Dependencies
import pandas as pd
import numpy as np
import tweepy
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import re
import csv
from textblob import TextBlob

In [2]:
from config import (consumer_key,consumer_secret,access_token,access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [3]:
# Target Search Term
target_terms = ("@DebtCrisisOrg", "@NatMortgageNews", "@CapitalOneCB",
                "@quickcarloannow","@USACreditdoctor","@creditrepair","@ForbesPersFin")


In [4]:
# Counter
# Variables for holding sentiments
sentiments = []

# Loop through all target users
for target in target_terms:
    counter = 1
#Loop through 100 tweets
    for x in range(5):

    #Get tweets from homefeed
        public_tweets = api.user_timeline(target, page=x,count=200)


        for tweet in public_tweets:
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            text=tweet['text']
            date=tweet['created_at']


 # Add sentiments for each tweet into a list
            sentiments.append({"Tweets":text,
                       "Positive": pos,
                       "Negative": neg})

# Add to counter 
            counter += 1


In [5]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiments)
sentiments_pd.shape


(6723, 3)

In [6]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [7]:
sentiments_pd['tidy_tweet'] = np.vectorize(remove_pattern)(sentiments_pd['Tweets'], "@[\w]*") 
sentiments_pd.shape

(6723, 4)

In [8]:
def f(row):
    if (row['Positive']>row['Negative']):
        val = "Positive"
        
    elif (row['Positive']<row['Negative']):
        val = "Negative"
    else:
        val = "Neutral"
    return val

In [9]:
sentiments_pd['Sentiment'] = sentiments_pd.apply(f, axis=1)


In [10]:
sentiments_pd['tidy_tweet'] = sentiments_pd['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [11]:
sentiments_pd

,Negative,Positive,Tweets,tidy_tweet,Sentiment
0,0.150,0.000,Samantha - a @DebtCrisisOrg team member - writ...,Samantha a team member writes about her f...,Negative
1,0.414,0.000,This is BAD for the #economy.\n\nhttps://t.co/...,This is BAD for the #economy https t co yg...,Negative
2,0.238,0.000,Last year was difficult for #students and #fam...,Last year was difficult for #students and #fam...,Negative
3,0.000,0.281,50 states. 50 governors. 1 call to create a #S...,states governors call to create a #S...,Positive
4,0.000,0.000,The people have spoken: time for lawmakers to ...,The people have spoken time for lawmakers to ...,Neutral
5,0.000,0.204,A great ally to #studentloan borrowers in #Wis...,A great ally to #studentloan borrowers in #Wis...,Positive
6,0.000,0.559,IMPORTANT INFORMATION.\n\nhttps://t.co/GrUU7VWKKG,IMPORTANT INFORMATION https t co GrUU VWKKG,Positive
7,0.000,0.000,Educated Americans can't afford to return to t...,Educated Americans can t afford to return to t...,Neutral
8,0.000,0.000,RT @DebtCrisisOrg: TUNE IN LIVE IN 1 HOUR: Pro...,RT TUNE IN LIVE IN HOUR Program director ...,Neutral
9,0.000,0.000,TUNE IN LIVE IN 1 HOUR: Program director @chou...,TUNE IN LIVE IN HOUR Program director join...,Neutral


In [13]:
new_df = sentiments_pd[sentiments_pd.Sentiment != 'Neutral']

In [14]:
finance_text=new_df.drop(['Positive','Negative','Tweets'],axis=1)


In [19]:
finance_text.shape

(4456, 2)

In [17]:
finance_text.to_csv("finance_text.txt", encoding="utf-8", sep=',',index=False)

In [20]:
finance_text.to_csv("finance_text.csv", encoding="utf-8", sep=',',index=False)

In [7]:
!jupyter nbconvert --to script financial.ipynb

[NbConvertApp] Converting notebook financial.ipynb to script
[NbConvertApp] Writing 3042 bytes to financial.py
